In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

os.getcwd()

'/Users/aneeshussain/Code/diss/analysis'

In [2]:
current_dir = os.getcwd()

parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
os.chdir(parent_dir)

sys.path.append(parent_dir)

print(f"Changed directory to: {os.getcwd()}")

Changed directory to: /Users/aneeshussain/Code/diss


In [154]:
# file_path = './data/all_exchanges.csv' 
file_path = './data/processed/binance.csv' 
df = pd.read_csv(file_path)

perp_df = df[df['contract'] == 'perpetual']

perp_df['time'] = pd.to_datetime(perp_df['time'])

daily_volume_perp = perp_df.groupby('time')['volume'].sum().reset_index()

daily_volume_perp['30_day_rolling_avg'] = daily_volume_perp['volume'].rolling(window=28).mean()

spot_df = df[df['contract'] == 'spot']

daily_volume_spot = spot_df.groupby('time')['volume'].sum().reset_index()

daily_volume_spot['30_day_rolling_avg'] = daily_volume_spot['volume'].rolling(window=28).mean()

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=daily_volume_spot['time'],
    y=daily_volume_spot['30_day_rolling_avg'],
    mode='lines',
    name='Spot',
    yaxis='y1',
    line=dict(width=2, color='white')
))

fig.add_trace(go.Scatter(
    x=daily_volume_spot['time'],
    y=(daily_volume_perp['30_day_rolling_avg'] / daily_volume_spot['30_day_rolling_avg']),
    mode='lines',
    name='Ratio',
    yaxis='y2',
    line=dict(width=1.5, color='grey'),
    opacity=0.7
))

fig.add_trace(go.Scatter(
    x=daily_volume_perp['time'],
    y=daily_volume_perp['30_day_rolling_avg'],
    mode='lines',
    name='Perpetual',
    yaxis='y1',
    line=dict(width=2, color='blue')
))


fig.update_layout(
    yaxis=dict(title="Daily Traded Perpetual Volume ($)", showgrid=True),
    yaxis2=dict(title='Perpetual to Spot Volume Ratio', overlaying='y', side='right', showgrid=False, ticksuffix='x'),
    template='plotly_white',
    hovermode='x unified',
    width=1200,
    height=400,
    legend=dict(orientation='h', x=0.5, y=-0.2, xanchor='center'),
    showlegend=False,
    margin=dict(l=50, r=50, t=20, b=50)
)

fig.update_xaxes(
    tickformat='%b %Y',
    tickangle=0,
    showgrid=True,
    zeroline=True,
    showticklabels=True,
    dtick='M3'
)

fig.show()

/var/folders/mf/b67_blxs1rj380h3rh2kgtlh0000gn/T/ipykernel_48492/2763254380.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [153]:
file_path = './data/processed/binance_btc_test.csv' 
df = pd.read_csv(file_path)

df['time'] = pd.to_datetime(df['time'])
march_df = df[(df['time'] >= '2024-03-01') & (df['time'] < '2024-03-14')]

spot_df = march_df[march_df['contract'] == 'spot'][['time', 'close']].rename(columns={'close': 'spot_price'})
perp_df = march_df[march_df['contract'] == 'perpetual'][['time', 'close']].rename(columns={'close': 'futures_price'})

merged_df = pd.merge(spot_df, perp_df, on='time')

merged_df['basis'] = merged_df['futures_price'] - merged_df['spot_price']

price_fig = go.Figure()

price_fig.add_trace(go.Scatter(
    x=merged_df['time'],
    y=merged_df['spot_price'],
    mode='lines',
    name='Spot',
    line=dict(width=1, color='blue'),
    opacity=0.7
))

price_fig.add_trace(go.Scatter(
    x=merged_df['time'],
    y=merged_df['futures_price'],
    mode='lines',
    name='Perpetual',
    line=dict(width=1, color='red'),
    opacity=0.7
))

price_fig.update_layout(
    yaxis=dict(title='Price ($)'),
    xaxis=dict(tickformat='%d.%m', tickmode='auto'),
    width=1200,
    height=400,
    template='plotly_white',
    legend=dict(orientation='h', x=0.5, y=-0.1, xanchor='center'),
    margin=dict(l=50, r=50, t=20, b=50)
)

price_fig.show()

basis_fig = go.Figure()

basis_fig.add_trace(go.Scatter(
    x=merged_df['time'],
    y=merged_df['basis'],
    mode='lines',
    name='Basis',
    line=dict(width=2, color='blue')
))

basis_fig.update_layout(
    yaxis=dict(title='Basis ($)', range=[0, max(merged_df['basis']) * 1.1]),
    xaxis=dict(tickformat='%d.%m', tickmode='auto'),
    width=1200,
    height=400,
    template='plotly_white',
    legend=dict(orientation='h', x=0.5, y=-0.05, xanchor='center'),
    margin=dict(l=50, r=50, t=20, b=50)
)

basis_fig.show()


In [150]:
file_path = './data/processed/binance_btc_test.csv'
df = pd.read_csv(file_path)

df['time'] = pd.to_datetime(df['time'])

perp_df = df[df['contract'] == 'perpetual'][['time', 'funding rate']]

perp_df['annualised_funding_rate'] = ((1 + perp_df['funding rate']) ** (3 * 365) - 1) * 100

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=perp_df['time'],
    y=perp_df['annualised_funding_rate'],
    mode='lines',
    name='Annualized Funding Rate',
    line=dict(width=1, color='blue')
))

fig.update_layout(
    xaxis=dict(tickformat='%d.%m'),
    yaxis=dict(title='Annualised Funding Rate (%)', zeroline=True),
    width=1200,
    height=400,
    template='plotly_white',
    margin=dict(l=50, r=50, t=20, b=50)
)

fig.update_xaxes(
    tickformat='%b %Y',
    tickangle=0,
    showgrid=True,
    zeroline=True,
    showticklabels=True,
    dtick='M3'
)

fig.show()

In [169]:
hstr = pd.read_csv(f'./results/hold/collateral_log.csv')
ststr = pd.read_csv(f'./results/simple_threshold/collateral_log.csv')
srstr = pd.read_csv(f'./results/simple_reinvest/collateral_log.csv')
ctstr = pd.read_csv(f'./results/complex_threshold/collateral_log.csv')
crstr = pd.read_csv(f'./results/complex_reinvest/collateral_log.csv')

initial_investment = 10000000

hstr['time'] = pd.to_datetime(hstr['time'])
hstr.set_index('time', inplace=True)

ststr['time'] = pd.to_datetime(ststr['time'])
ststr.set_index('time', inplace=True)

srstr['time'] = pd.to_datetime(srstr['time'])
srstr.set_index('time', inplace=True)

ctstr['time'] = pd.to_datetime(ctstr['time'])
ctstr.set_index('time', inplace=True)

crstr['time'] = pd.to_datetime(crstr['time'])
crstr.set_index('time', inplace=True)

collateral_columns = [
    'binance_btc_collateral', 'binance_eth_collateral', 'binance_liquid_cash',
    'okx_btc_collateral', 'okx_eth_collateral', 'okx_liquid_cash',
    'bybit_btc_collateral', 'bybit_eth_collateral', 'bybit_liquid_cash'
]

funding_columns = ['binance_funding', 'okx_funding', 'bybit_funding']

unrealised_pnl_columns = ['binance_unrealised_pnl', 'okx_unrealised_pnl', 'bybit_unrealised_pnl']

hstr['pnl_basis'] = hstr[collateral_columns].sum(axis=1) - initial_investment
hstr['pnl_funding'] = hstr[funding_columns].sum(axis=1)
hstr['total_pnl'] = hstr['pnl_basis'] + hstr['pnl_funding']

hstr['cumulative_total_pnl_pct'] = (hstr['total_pnl'] / initial_investment) * 100


ststr['pnl_basis'] = ststr[collateral_columns].sum(axis=1) - initial_investment
ststr['pnl_funding'] = ststr[funding_columns].sum(axis=1)
ststr['total_pnl'] = ststr['pnl_basis'] + ststr['pnl_funding']

ststr['cumulative_total_pnl_pct'] = (ststr['total_pnl'] / initial_investment) * 100


srstr['pnl_basis'] = srstr[collateral_columns].sum(axis=1) - initial_investment
srstr['pnl_funding'] = srstr[funding_columns].sum(axis=1)
srstr['total_pnl'] = srstr['pnl_basis'] + srstr['pnl_funding']

srstr['cumulative_total_pnl_pct'] = (srstr['total_pnl'] / initial_investment) * 100


ctstr['pnl_basis'] = ctstr[collateral_columns].sum(axis=1) - initial_investment
ctstr['pnl_funding'] = ctstr[funding_columns].sum(axis=1)
ctstr['total_pnl'] = ctstr['pnl_basis'] + ctstr['pnl_funding']

ctstr['cumulative_total_pnl_pct'] = (ctstr['total_pnl'] / initial_investment) * 100


crstr['pnl_basis'] = crstr[collateral_columns].sum(axis=1) - initial_investment
crstr['pnl_funding'] = crstr[funding_columns].sum(axis=1)
crstr['total_pnl'] = crstr['pnl_basis'] + crstr['pnl_funding']

crstr['cumulative_total_pnl_pct'] = (crstr['total_pnl'] / initial_investment) * 100

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=hstr.index,
    y=hstr['total_pnl'],
    mode='lines',
    name='Hold',
    line=dict(width=1, color='blue'),
    hovertemplate='Hold P&L: $%{y:.2f}<br>Date: %{x}',
    yaxis='y1'
))

fig.add_trace(go.Scatter(
    x=hstr.index,
    y=hstr['cumulative_total_pnl_pct'],
    mode='lines',
    name='Hold',
    line=dict(width=1, color='blue'),
    hovertemplate='Hold P&L: %{y:.2f}%<br>Date: %{x}',
    yaxis='y2',
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=ststr.index,
    y=ststr['total_pnl'],
    mode='lines',
    name='Threshold',
    line=dict(width=1, color='orange'),
    hovertemplate='Simple Threshold P&L: $%{y:.2f}<br>Date: %{x}',
    yaxis='y1'
))

fig.add_trace(go.Scatter(
    x=ststr.index,
    y=ststr['cumulative_total_pnl_pct'],
    mode='lines',
    name='Threshold',
    line=dict(width=1, color='orange'),
    hovertemplate='Simple Threshold P&L: %{y:.2f}%<br>Date: %{x}',
    yaxis='y2',
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=srstr.index,
    y=srstr['total_pnl'],
    mode='lines',
    name='Threshold & Reinvesting',
    line=dict(width=1, color='purple'),
    hovertemplate='Simple Reinvest P&L: $%{y:.2f}<br>Date: %{x}',
    yaxis='y1'
))

fig.add_trace(go.Scatter(
    x=srstr.index,
    y=srstr['cumulative_total_pnl_pct'],
    mode='lines',
    name='Threshold & Reinvesting',
    line=dict(width=1, color='purple'),
    hovertemplate='Simple Reinvest P&L: %{y:.2f}%<br>Date: %{x}',
    yaxis='y2',
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=ctstr.index,
    y=ctstr['total_pnl'],
    mode='lines',
    name='Dynamic',
    line=dict(width=1, color='green'),
    hovertemplate='Complex Threshold P&L: $%{y:.2f}<br>Date: %{x}',
    yaxis='y1'
))

fig.add_trace(go.Scatter(
    x=ctstr.index,
    y=ctstr['cumulative_total_pnl_pct'],
    mode='lines',
    name='Dynamic',
    line=dict(width=1, color='green'),
    hovertemplate='Complex Threshold P&L: %{y:.2f}%<br>Date: %{x}',
    yaxis='y2',
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=crstr.index,
    y=crstr['total_pnl'],
    mode='lines',
    name='Dynamic & Reinvesting',
    line=dict(width=1, color='red'),
    hovertemplate='Complex Reinvest P&L: $%{y:.2f}<br>Date: %{x}',
    yaxis='y1'
))

fig.add_trace(go.Scatter(
    x=crstr.index,
    y=crstr['cumulative_total_pnl_pct'],
    mode='lines',
    name='Dynamic & Reinvesting',
    line=dict(width=1, color='red'),
    hovertemplate='Complex Reinvest P&L: %{y:.2f}%<br>Date: %{x}',
    yaxis='y2',
    showlegend=False
))

fig.update_layout(
    yaxis=dict(title='P&L ($)'),
    yaxis2=dict(title='P&L (%)', overlaying='y', side='right'),
    template='plotly_white',
    hovermode='x unified',
    legend=dict(
        orientation='h',
        x=0.5,
        y=-0.1,
        xanchor='center',
        yanchor='top'
    ),
    width=1200,
    height=400,
    margin=dict(l=50, r=50, t=20, b=50)
)

fig.update_xaxes(
    tickformat='%b %Y',
    tickangle=0,
    showgrid=True,
    zeroline=True,
    showticklabels=True,
    dtick='M3'
)

fig.show()

In [168]:
cutoff_date = '2024-01-01'

hstr = hstr[hstr.index < cutoff_date]
ststr = ststr[ststr.index < cutoff_date]
srstr = srstr[srstr.index < cutoff_date]
ctstr = ctstr[ctstr.index < cutoff_date]
crstr = crstr[crstr.index < cutoff_date]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=hstr.index,
    y=hstr['total_pnl'],
    mode='lines',
    name='Hold',
    line=dict(width=1, color='blue'),
    hovertemplate='Hold P&L: $%{y:.2f}<br>Date: %{x}',
    yaxis='y1'
))

fig.add_trace(go.Scatter(
    x=hstr.index,
    y=hstr['cumulative_total_pnl_pct'],
    mode='lines',
    name='Hold',
    line=dict(width=1, color='blue'),
    hovertemplate='Hold P&L: %{y:.2f}%<br>Date: %{x}',
    yaxis='y2',
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=ststr.index,
    y=ststr['total_pnl'],
    mode='lines',
    name='Threshold',
    line=dict(width=1, color='orange'),
    hovertemplate='Simple Threshold P&L: $%{y:.2f}<br>Date: %{x}',
    yaxis='y1'
))

fig.add_trace(go.Scatter(
    x=ststr.index,
    y=ststr['cumulative_total_pnl_pct'],
    mode='lines',
    name='Threshold',
    line=dict(width=1, color='orange'),
    hovertemplate='Simple Threshold P&L: %{y:.2f}%<br>Date: %{x}',
    yaxis='y2',
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=ctstr.index,
    y=ctstr['total_pnl'],
    mode='lines',
    name='Dynamic',
    line=dict(width=1, color='green'),
    hovertemplate='Complex Threshold P&L: $%{y:.2f}<br>Date: %{x}',
    yaxis='y1'
))

fig.add_trace(go.Scatter(
    x=ctstr.index,
    y=ctstr['cumulative_total_pnl_pct'],
    mode='lines',
    name='Dynamic',
    line=dict(width=1, color='green'),
    hovertemplate='Complex Threshold P&L: %{y:.2f}%<br>Date: %{x}',
    yaxis='y2',
    showlegend=False
))

fig.update_layout(
    yaxis=dict(title='P&L ($)'),
    yaxis2=dict(title='P&L (%)', overlaying='y', side='right'),
    template='plotly_white',
    hovermode='x unified',
    legend=dict(
        orientation='h',
        x=0.5,
        y=-0.1,
        xanchor='center',
        yanchor='top'
    ),
    width=1200,
    height=400,
    margin=dict(l=50, r=50, t=20, b=50)
)

fig.update_xaxes(
    tickformat='%b %Y',
    tickangle=0,
    showgrid=True,
    zeroline=True,
    showticklabels=True,
    dtick='M3'
)

fig.show()

In [162]:
simple_file = './results/simple_threshold/yield.csv'
simple_reinvest_file = './results/simple_reinvest/yield.csv'
complex_file = './results/complex_threshold/yield.csv'
complex_reinvest_file = './results/complex_reinvest/yield.csv'

simple_df = pd.read_csv(simple_file)
simple_df['date'] = pd.to_datetime(simple_df['date'])
simple_df.set_index('date', inplace=True)

simple_reinvest_df = pd.read_csv(simple_reinvest_file)
simple_reinvest_df['date'] = pd.to_datetime(simple_reinvest_df['date'])
simple_reinvest_df.set_index('date', inplace=True)

complex_df = pd.read_csv(complex_file)
complex_df['date'] = pd.to_datetime(complex_df['date'])
complex_df.set_index('date', inplace=True)

complex_reinvest_df = pd.read_csv(complex_reinvest_file)
complex_reinvest_df['date'] = pd.to_datetime(complex_reinvest_df['date'])
complex_reinvest_df.set_index('date', inplace=True)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=complex_df.index,
    y=complex_df['total_yield'],
    mode='lines',
    name='Dynamic',
    line=dict(width=2, color='orange'),
    opacity=1.0,
    legendrank=2
))

fig.add_trace(go.Scatter(
    x=simple_df.index,
    y=simple_df['total_yield'],
    mode='lines',
    name='Threshold',
    line=dict(width=2, color='blue'),
    opacity=1.0,
    legendrank=1
))

fig.update_layout(
    yaxis_title='Average Perpetual Yield (%)',
    template='plotly_white',
    hovermode='x unified',
    legend=dict(
        orientation='h',
        x=0.5,
        y=-0.1,
        xanchor='center',
        yanchor='top'
    ),
    width=1200,
    height=400,
    margin=dict(l=50, r=50, t=20, b=50)
)

fig.update_xaxes(
    tickformat='%b %Y',
    tickangle=0,
    showgrid=True,
    zeroline=True,
    showticklabels=True,
    dtick='M3'
)

fig.show()